In [2]:
import gc
import time
import numpy as np
import pandas as pd
from datetime import datetime

In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
merchant = pd.read_csv('merchants.csv')
new_trans = pd.read_csv('new_merchant_transactions.csv')
history_trans = pd.read_csv('historical_transactions.csv')

In [4]:
def convert_object(se): #dictinalry encoder
    value = se.unique().tolist()
    value.sort()
    return se.map(pd.Series(range(len(value)),index = value)).values

In [6]:
se_map = convert_object(train['first_active_month'].append(test['first_active_month'].astype(str)))

In [7]:
train['first_active_month'] = se_map[:train.shape[0]]
test['first_active_month'] = se_map[train.shape[0]:]

In [11]:
train.to_csv('preprocess/train_pre.csv',index = False)
test.to_csv('preprocess/test_pre.csv',index = False)

In [10]:
history_trans.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29112361 entries, 0 to 29112360
Data columns (total 14 columns):
authorized_flag         object
card_id                 object
city_id                 int64
category_1              object
installments            int64
category_3              object
merchant_category_id    int64
merchant_id             object
month_lag               int64
purchase_amount         float64
purchase_date           object
category_2              float64
state_id                int64
subsector_id            int64
dtypes: float64(2), int64(6), object(6)
memory usage: 12.7 GB


In [12]:
del train
del test
gc.collect()

179

In [3]:
merchant = pd.read_csv('merchants.csv')
new_trans = pd.read_csv('new_merchant_transactions.csv')
history_trans = pd.read_csv('historical_transactions.csv')

In [6]:
mch_ctgr_cols = ['category_1', 'most_recent_sales_range', 'most_recent_purchases_range', 'category_4']
ctgr_cols = ['merchant_id', 'merchant_group_id', 'merchant_category_id',
       'subsector_id', 'category_1',
       'most_recent_sales_range', 'most_recent_purchases_range',
       'category_4', 'city_id', 'state_id', 'category_2']
nmrc_cols = ['numerical_1', 'numerical_2',
     'avg_sales_lag3', 'avg_purchases_lag3', 'active_months_lag3',
       'avg_sales_lag6', 'avg_purchases_lag6', 'active_months_lag6',
       'avg_sales_lag12', 'avg_purchases_lag12', 'active_months_lag12']

In [7]:
for col in mch_ctgr_cols:
    merchant[col] = convert_object(merchant[col])

In [8]:
merchant[ctgr_cols] = merchant[ctgr_cols].fillna(-1) #mark na with -1

In [13]:
inf_cols = ['avg_purchases_lag3', 'avg_purchases_lag6', 'avg_purchases_lag12']
merchant[inf_cols] = merchant[inf_cols].replace(np.inf, merchant[inf_cols].replace(np.inf, -99).max()) # replace inf with max

In [14]:
merchant[nmrc_cols].describe()

,numerical_1,numerical_2,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12
count,334696.000000,334696.000000,334683.000000,334696.000000,334696.000000,3.346830e+05,334696.000000,334696.000000,3.346830e+05,334696.000000,334696.000000
mean,0.011476,0.008103,13.832993,2.145143,2.994108,2.165079e+01,2.390194,5.947397,2.522771e+01,2.529277,11.599335
std,1.098154,1.070497,2395.489999,213.955844,0.095247,3.947108e+03,194.324264,0.394936,5.251842e+03,174.419034,1.520138
min,-0.057471,-0.057471,-82.130000,0.333495,1.000000,-8.213000e+01,0.167045,1.000000,-8.213000e+01,0.098330,1.000000
25%,-0.057471,-0.057471,0.880000,0.923650,3.000000,8.500000e-01,0.902247,6.000000,8.500000e-01,0.898333,12.000000
50%,-0.057471,-0.057471,1.000000,1.016667,3.000000,1.010000e+00,1.026961,6.000000,1.020000e+00,1.043361,12.000000
75%,-0.047556,-0.047556,1.160000,1.146522,3.000000,1.230000e+00,1.215575,6.000000,1.290000e+00,1.266480,12.000000
max,183.735111,182.079322,851844.640000,61851.333333,3.000000,1.513959e+06,56077.500000,6.000000,2.567408e+06,50215.555556,12.000000


In [15]:
merchant[nmrc_cols] = merchant[nmrc_cols].fillna(merchant[nmrc_cols].mean())

In [16]:
merchant[nmrc_cols].describe()

,numerical_1,numerical_2,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12
count,334696.000000,334696.000000,334696.000000,334696.000000,334696.000000,3.346960e+05,334696.000000,334696.000000,3.346960e+05,334696.000000,334696.000000
mean,0.011476,0.008103,13.832993,2.145143,2.994108,2.165079e+01,2.390194,5.947397,2.522771e+01,2.529277,11.599335
std,1.098154,1.070497,2395.443476,213.955844,0.095247,3.947031e+03,194.324264,0.394936,5.251740e+03,174.419034,1.520138
min,-0.057471,-0.057471,-82.130000,0.333495,1.000000,-8.213000e+01,0.167045,1.000000,-8.213000e+01,0.098330,1.000000
25%,-0.057471,-0.057471,0.880000,0.923650,3.000000,8.500000e-01,0.902247,6.000000,8.500000e-01,0.898333,12.000000
50%,-0.057471,-0.057471,1.000000,1.016667,3.000000,1.010000e+00,1.026961,6.000000,1.020000e+00,1.043361,12.000000
75%,-0.047556,-0.047556,1.160000,1.146522,3.000000,1.230000e+00,1.215575,6.000000,1.290000e+00,1.266480,12.000000
max,183.735111,182.079322,851844.640000,61851.333333,3.000000,1.513959e+06,56077.500000,6.000000,2.567408e+06,50215.555556,12.000000


In [35]:
duplicate_cols = []
for col in list(merchant.columns):
    if col in list(new_trans.columns):
        duplicate_cols.append(col)
        
duplicate_cols

['merchant_id',
 'merchant_category_id',
 'subsector_id',
 'category_1',
 'city_id',
 'state_id',
 'category_2']

In [37]:
merchant = merchant.drop(duplicate_cols[1:], axis = 1)


KeyError: "['merchant_category_id' 'subsector_id' 'category_1' 'city_id' 'state_id'\n 'category_2'] not found in axis"

In [38]:
merchant = merchant.loc[merchant['merchant_id'].drop_duplicates().index.tolist()].reset_index(drop = True)

In [43]:
transactions = pd.concat([new_trans, history_trans],axis = 0, ignore_index=True)

In [44]:
del new_trans
del history_trans

In [45]:
gc.collect()

979

In [46]:
numeric_cols = [ 'installments', 'month_lag', 'purchase_amount']
category_cols = ['authorized_flag', 'card_id', 'city_id', 'category_1',
       'category_3', 'merchant_category_id', 'merchant_id', 'category_2', 'state_id',
       'subsector_id']
time_cols = ['purchase_date']

In [48]:
for col in ['authorized_flag', 'category_1', 'category_3']:
    transactions[col] = convert_object(transactions[col].fillna(-1).astype(str))

In [49]:
transactions[category_cols] = transactions[category_cols].fillna(-1)
transactions['category_2'] = transactions['category_2'].astype(int)

In [51]:
transactions['purchase_month'] = transactions['purchase_date'].apply(lambda x:'-'.join(x.split(' ')[0].split('-')[:2]))
transactions['purchase_hour_section'] = transactions['purchase_date'].apply(lambda x: x.split(' ')[1].split(':')[0]).astype(int)//6
transactions['purchase_day'] = transactions['purchase_date'].apply(lambda x: datetime.strptime(x.split(" ")[0], "%Y-%m-%d").weekday())//5                                                                    
del transactions['purchase_date']

In [52]:
transactions['purchase_month'] = convert_object(transactions['purchase_month'].fillna(-1).astype(str))

In [53]:
cols = ['merchant_id', 'most_recent_sales_range', 'most_recent_purchases_range', 'category_4']
transactions = pd.merge(transactions, merchant[cols], how = 'left', on = 'merchant_id')

In [54]:
transactions.columns

Index(['authorized_flag', 'card_id', 'city_id', 'category_1', 'installments',
       'category_3', 'merchant_category_id', 'merchant_id', 'month_lag',
       'purchase_amount', 'category_2', 'state_id', 'subsector_id',
       'purchase_month', 'purchase_hour_section', 'purchase_day',
       'most_recent_sales_range', 'most_recent_purchases_range', 'category_4'],
      dtype='object')

In [59]:
numeric_cols = ['purchase_amount', 'installments']

category_cols = ['authorized_flag', 'city_id', 'category_1',
       'category_3', 'merchant_category_id','month_lag','most_recent_sales_range',
                 'most_recent_purchases_range', 'category_4',
                 'purchase_month', 'purchase_hour_section', 'purchase_day']

id_cols = ['card_id', 'merchant_id']

transactions[cols[1:]] = transactions[cols[1:]].fillna(-1).astype(int)
transactions[category_cols] =transactions[category_cols].fillna(-1).astype(str)

MemoryError: 

In [60]:
transactions.describe()

,authorized_flag,city_id,category_1,installments,category_3,merchant_category_id,month_lag,purchase_amount,category_2,state_id,subsector_id,purchase_month,purchase_hour_section,purchase_day,most_recent_sales_range,most_recent_purchases_range,category_4
count,3.107539e+07,3.107539e+07,3.107539e+07,3.107539e+07,3.107539e+07,3.107539e+07,3.107539e+07,3.107539e+07,3.107539e+07,3.107539e+07,3.107539e+07,3.107539e+07,3.107539e+07,3.107539e+07,3.107539e+07,3.107539e+07,3.107539e+07
mean,9.190064e-01,1.296453e+02,6.909406e-02,6.506728e-01,1.523464e+00,4.778518e+02,-4.110560e+00,-7.032267e-04,1.910568e+00,1.058662e+01,2.679329e+01,8.043680e+00,1.939643e+00,2.816023e-01,1.695518e+00,1.623897e+00,5.008945e-01
std,2.728253e-01,1.040926e+02,2.536140e-01,2.734985e+00,6.267563e-01,2.494821e+02,3.766495e+00,1.087456e+03,1.721771e+00,6.347145e+00,9.723248e+00,3.967029e+00,8.849273e-01,4.497805e-01,1.353224e+00,1.331830e+00,5.104890e-01
min,0.000000e+00,-1.000000e+00,0.000000e+00,-1.000000e+00,0.000000e+00,-1.000000e+00,-1.300000e+01,-7.469078e-01,-1.000000e+00,-1.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00
25%,1.000000e+00,5.300000e+01,0.000000e+00,0.000000e+00,1.000000e+00,3.020000e+02,-7.000000e+00,-7.199051e-01,1.000000e+00,9.000000e+00,1.900000e+01,5.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.000000e+00,9.000000e+01,0.000000e+00,0.000000e+00,1.000000e+00,4.540000e+02,-3.000000e+00,-6.875230e-01,1.000000e+00,9.000000e+00,2.900000e+01,9.000000e+00,2.000000e+00,0.000000e+00,2.000000e+00,2.000000e+00,1.000000e+00
75%,1.000000e+00,2.120000e+02,0.000000e+00,1.000000e+00,2.000000e+00,7.050000e+02,-1.000000e+00,-6.008651e-01,3.000000e+00,1.600000e+01,3.400000e+01,1.100000e+01,3.000000e+00,1.000000e+00,3.000000e+00,3.000000e+00,1.000000e+00
max,1.000000e+00,3.470000e+02,1.000000e+00,9.990000e+02,3.000000e+00,8.910000e+02,2.000000e+00,6.010604e+06,5.000000e+00,2.400000e+01,4.100000e+01,1.500000e+01,3.000000e+00,1.000000e+00,4.000000e+00,4.000000e+00,1.000000e+00


In [61]:
transactions.to_csv('preprocess/transactions_d_pre.csv', index = False)

In [62]:
del transactions
gc.collect()

634